In [23]:
%load_ext autoreload
%autoreload 2
import sys, os, socket
os.getcwd(), socket.gethostname()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


('/home/ajasja/projects/truncator', 'dig33')

In [24]:
import numpy
import numpy as np

from glob import glob
import os

import truncator

import pymol
from pymol import cmd

In [25]:
pdb_files = glob("out/R3/01_cut/*/*.pdb");


In [47]:
pick_name = "out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47_grC-ABD"

split_name = pick_name.split("__")
base_name = split_name[0]
group_name = split_name[-1].replace('gr','')
gsplit = group_name.split("-");
gsplit[0]=str(len(gsplit[0]))
gsplit[1]=str(len(gsplit[1]))
topology=gsplit[0]+gsplit[1]
#remove gr
struct_name=pick_name[:-9]
base_name,group_name,topology

#TODO figure out all connections acording to UP/DOWN status of helices
{'A-BCD':"[A,B+C+D] [A,D+C+B]".split(), 
 'AB-CD':"[A+B,C+D] [B+A,C+D] [A+B,D+C] [B+A,D+C] [C+D,A+B] [C+D,B+A] [D+C,A+B] [D+C,B+A]".split(), 
 'AD-BC':"[A+D,C+B] [D+A,C+B] [A+D,B+C] [D+A,B+C] [C+B,A+D] [C+B,D+A] [B+C,A+D] [B+C,D+A]".split(), 
 'B-ACD':"[B,A+C+D] [B,D+C+A]".split(), 
 'C-ABD':"[C,B+A+D] [C,D+A+B]".split(), 
 'D-ABC':"[D,A+B+C] [D,C+A+B]".split()};


In [50]:
struct_name=truncator.find_input(struct_name, pdb_files); struct_name

'out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb'

In [28]:
%%time
reloop_result = truncator.reloop(struct_name, "out/R3/TEST02", 
                          script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml', 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          extra_flags="-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm",
                          structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
                          chain_connections='[A,D+C+B]',
                          loopLengthRange="2,5",
                          resAdjustmentRangeSide1="-3,1", 
                          resAdjustmentRangeSide2="-3,1",
                          RMSthreshold='.5',
                          test_run=False, verbose=False, add_suffix=True)

print(reloop_result.score_file)

ZCON_37__numH4.5__from-21.21__to22.24__cc[A,D+C+B]__llr2,5__ar1-3,1__ar2-3,1__rms.5.sc
CPU times: user 2min 17s, sys: 13.2 s, total: 2min 30s
Wall time: 6min 18s


In [35]:
import dask, distributed
client = distributed.Client(scheduler_file='scheduler.json'); client

Client Scheduler: tcp://172.16.131.63:29303 Dashboard: http://172.16.131.63:8787/status,Cluster Workers: 2 Cores: 2 Memory: 4.00 GB


In [42]:
pdb_files = glob("out/R3/01_cut/*/*.pdb");
regrouped_files = glob("out/R3/02_regruped/*/*.pdb"); 
picks = truncator.read_file_lines("out/R3/Reloop.picks", skip_comments=True)
for pick_name in picks:
    struct_name=pick_name[:-9]
    struct_name=truncator.find_input(struct_name, pdb_files)
    full_pick_name=truncator.find_input(pick_name, regrouped_files)
    print(f"load {full_pick_name}")
    print(f"load {struct_name}")
    print("")
        
   
  

load out/R3/02_regruped/4.5H/ZCON_37__numH4.5__from-21.21__to22.24__grAB-CD.pdb
load out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb

load out/R3/02_regruped/4.5H/ZCON_37__numH4.5__from-21.21__to22.24__grA-BCD.pdb
load out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb

load out/R3/02_regruped/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__grAD-BC.pdb
load out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb

load out/R3/02_regruped/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__grC-ABD.pdb
load out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb

load out/R3/02_regruped/4.5H/ZCON_127__numH4.5__from-20.77__to21.47__grAD-BC.pdb
load out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb

load out/R3/02_regruped/4.5H/ZCON_127__numH4.5__from-20.77__to21.47__grC-ABD.pdb
load out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb

load out/R3/02_regruped/4.5H/ZCON_13__numH4.5__from-17.52__to22.91__grAD-BC.pdb
load o

In [43]:
#TODO figure out all connections acording to UP/DOWN status of helices
gr_map= \
{'A-BCD':"[A,B+C+D] [A,D+C+B]".split(), 
 'AB-CD':"[A+B,C+D] [B+A,C+D] [A+B,D+C] [B+A,D+C] [C+D,A+B] [C+D,B+A] [D+C,A+B] [D+C,B+A]".split(), 
 'AD-BC':"[A+D,C+B] [D+A,C+B] [A+D,B+C] [D+A,B+C] [C+B,A+D] [C+B,D+A] [B+C,A+D] [B+C,D+A]".split(), 
 'B-ACD':"[B,A+C+D] [B,D+C+A]".split(), 
 'C-ABD':"[C,B+A+D] [C,D+A+B]".split(), 
 'D-ABC':"[D,A+B+C] [D,C+A+B]".split()};


In [44]:
list(range(-2,1))

[-2, -1, 0]

In [45]:
n = 0
res = []
#pick_name = "ZCON_37__numH4.5__from-21.21__to22.24__grAB-CD"
for pick_name in picks:
    split_name = pick_name.split("__")
    base_name = split_name[0]
    group_name = split_name[-1].replace('gr','')
    gsplit = group_name.split("-");
    gsplit[0]=str(len(gsplit[0]))
    gsplit[1]=str(len(gsplit[1]))
    topology=gsplit[0]+gsplit[1]
    #remove gr
    struct_name=pick_name[:-9]
    struct_name=truncator.find_input(struct_name, pdb_files)

    print (base_name, group_name, struct_name, gr_map[group_name])
    sn = 0;


    for chain_connections in gr_map[group_name]:
        for loopLengthRange in '4 5'.split():
            for ar1 in range(-3,0):
                for ar2 in range(-3,0):
                    n+=1; sn+=1;
                    #print(close_type, loop_length, start,end)
                    res.append( client.submit(
                        truncator.reloop, struct_name, "out/R3/08_relooped/"+base_name,
                              script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml',
                              rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                              structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
                              chain_connections=chain_connections,
                              loopLengthRange="2,"+loopLengthRange,
                              resAdjustmentRangeSide1=f"{ar1},0",
                              resAdjustmentRangeSide2=f"{ar2},0",
                              RMSthreshold=0.5,
                              test_run=False, verbose=True, add_suffix=True, skip_existing=False)
                        )


print(n)

ZCON_37 AB-CD out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb ['[A+B,C+D]', '[B+A,C+D]', '[A+B,D+C]', '[B+A,D+C]', '[C+D,A+B]', '[C+D,B+A]', '[D+C,A+B]', '[D+C,B+A]']
ZCON_37 A-BCD out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb ['[A,B+C+D]', '[A,D+C+B]']
ZCON_15_re_loop AD-BC out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A,B+C]', '[C+B,A+D]', '[C+B,D+A]', '[B+C,A+D]', '[B+C,D+A]']
ZCON_15_re_loop C-ABD out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb ['[C,B+A+D]', '[C,D+A+B]']
ZCON_127 AD-BC out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A,B+C]', '[C+B,A+D]', '[C+B,D+A]', '[B+C,A+D]', '[B+C,D+A]']
ZCON_127 C-ABD out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb ['[C,B+A+D]', '[C,D+A+B]']
ZCON_13 AD-BC out/R3/01_cut/4.5H/ZCON_13__numH4.5__from-17.52__to22.91.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A

In [46]:
distributed.wait(res, timeout=5);

TimeoutError: Timeout

In [15]:
client.cancel(res)

In [19]:
res[0].result()

'out/R3/04_relooped/ZCON_37/ZCON_37__numH4.5__from-21.21__to22.24__cc[A+B,C+D]__llr2,4__ar1-3__ar2-3__rms0.5.log'